In [1]:
VERSION = '01_v1_36'
MODEL = 'microsoft/deberta-v3-base'
#'microsoft/deberta-base'
#'microsoft/deberta-v3-large'
LR = 2e-5 #8e-6
HEAD_LR = 2e-4 #2e-5 #8e-6 
SEED = 100
TRN_BS = 8
VAL_BS = 8
ACCUM_STEP = 1
HIDDEN_DROP_PROB = 0
P_DROP = 0
RNN = 'none'
WARMUP_RATIO = 0.1
HEAD = 'simple'
AUG = 'false'
MIXUP_ALPHA = 0
P_AUG = 0
AUG_STOP_EPOCH = 0
MSD = 'false'
MULTI_LAYERS = 1
EVAL_STEP_START_EPOCH = 3
EVAL_STEP = -1
NUM_LABELS = 6
NUM_LABELS_2 = -1
FP16 = 'true'
WD = 0.01
FREEZE = 'false'
MULTI_TASK = 'false' 
W_MT = 0 
AWP = 'true' #'false'
AWP_LR = 1e-4
AWP_EPS = 1e-4
AWP_START_EPOCH = 1
#PRETRAINED_DETECTOR_PATH = f'../../input/tascj/result/deberta_base_fold0.pth'
PRETRAINED_DETECTOR_PATH = 'none' #f'../../05_Detection/exp/result/05_v2_09/model_seed100_fold0_swa.pth'
MASK_PROB = 0 #0.5
MASK_RATIO = 0 #0.1
SCHEDULER = 'cosine_hard'
CP = 'false'
WINDOW_SIZE = -1 #512
INNER_LEN = -1 #384
EDGE_LEN = -1 #64
MAX_LEN = 512

GRAD_CLIP = 1

LOSS = 'smoothl1'#'mse'

In [2]:
FOLD = 0
INPUT_PATH = '../../input/feedback-prize-english-language-learning/'
FOLD_PATH = '../../00_EDA/00_v1_02/result/'
MODE = 'train'
EPOCHS = 5
STOP_EPOCH = 5
RESTART = 1
NUM_CYCLES = EPOCHS
PRETRAIN_PATH = 'none' #f'result/{VERSION}/model_seed{SEED}_fold{FOLD}_epoch{EPOCHS}_pseudo.pth'

!python ../$VERSION/train.py --model $MODEL --version $VERSION --fold_path $FOLD_PATH --fold $FOLD --seed $SEED \
--lr $LR --head_lr $HEAD_LR --trn_batch_size $TRN_BS --val_batch_size $VAL_BS \
--epochs $EPOCHS --hidden_drop_prob $HIDDEN_DROP_PROB --p_drop $P_DROP \
--accumulate_grad_batches $ACCUM_STEP --rnn $RNN --warmup_ratio $WARMUP_RATIO --loss $LOSS --aug $AUG --head $HEAD \
--mixup_alpha $MIXUP_ALPHA --p_aug $P_AUG --aug_stop_epoch $AUG_STOP_EPOCH \
--msd $MSD --multi_layers $MULTI_LAYERS \
--eval_step_start_epoch $EVAL_STEP_START_EPOCH --eval_step $EVAL_STEP --stop_epoch $STOP_EPOCH \
--num_labels $NUM_LABELS --num_labels_2 $NUM_LABELS_2 \
--restart_epoch $RESTART --fp16 $FP16 --weight_decay $WD --freeze_layers $FREEZE \
--mt $MULTI_TASK --w_mt $W_MT \
--awp $AWP --awp_lr $AWP_LR --awp_eps $AWP_EPS --awp_start_epoch $AWP_START_EPOCH \
--pretrained_detector_path $PRETRAINED_DETECTOR_PATH --mask_prob $MASK_PROB --mask_ratio $MASK_RATIO \
--scheduler $SCHEDULER --num_cycles $NUM_CYCLES --check_pointing $CP \
--window_size $WINDOW_SIZE --inner_len $INNER_LEN --edge_len $EDGE_LEN \
--gradient_clip_val $GRAD_CLIP \
--input_path $INPUT_PATH --mode $MODE --pretrain_path $PRETRAIN_PATH --max_length $MAX_LEN

torch 1.10.1
args.mode =  train
train_df.shape =  (3911, 8)
load folds...
trn_df.shape =  (3128, 8)
val_df.shape =  (783, 8)
2022-09-16 08:45:16.155213: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.2/lib64:
2022-09-16 08:45:16.155237: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
self.warmup_ratio =  0.1
self.num_train_steps =  1955
num_warmup_steps =  195
lr :  [0.0, 0.0, 0.0]
100%|███████████████████████████████████████████| 98/98 [00:22<00:00,  4.36it/s]
epoch 1: trn_loss = 0.7410, val_loss = 0.1128, trn_score = 1.0394, val_score = 0.4760
model (best score) saved
lr :  [8.224468037456969e-06, 8.224468037456969e-06, 8.224468037456969e-05]
100%|███████████████████████████████████████████| 98/98 [00:22<00:00,  4.

# OOF

In [ ]:
FOLD = 0
INPUT_PATH = '../../input/feedback-prize-english-language-learning/'
FOLD_PATH = '../../00_EDA/00_v1_02/result/'
WEIGHT_PATH = f'./result/{VERSION}/model_seed{SEED}_fold{FOLD}.pth'

!python ../$VERSION/validation.py --model $MODEL --version $VERSION \
--fold_path $FOLD_PATH --fold $FOLD --seed $SEED --val_batch_size $VAL_BS \
--rnn $RNN --loss $LOSS --mt $MULTI_TASK --num_labels $NUM_LABELS --loss $LOSS --weight_path $WEIGHT_PATH \
--window_size $WINDOW_SIZE --inner_len $INNER_LEN --edge_len $EDGE_LEN --max_length $MAX_LEN

In [ ]:
import pandas as pd

num_fold = 5

preds = []
for fold in range(num_fold):
    tmp_df = pd.read_csv(f'./result/{VERSION}/pred_fold{fold}.csv')
    preds.append(tmp_df)
    
pred_df = pd.concat(preds, axis=0).reset_index(drop=True)
train_df = pd.read_csv('../../input/feedback-prize-english-language-learning/train.csv')

oof_df = train_df[['text_id']].merge(pred_df, on='text_id', how='left')
oof_df.to_csv(f'./result/{VERSION}/oof_df.csv', index=False)

In [ ]:
oof_df.shape, oof_df.isnull().sum()